In [1]:
# Author: Mike Hagenow
# Generate some templated behaviors for sanding (including corrections) to be learned through DMP-LWR

import numpy as np
import rospy

from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import HybridSegment
from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import DMPLWRhardcoded
from core_robotics.PyBSpline import BSplineSurface
from scipy.spatial.transform import Rotation as ScipyR
from scipy.spatial.transform import Slerp

# starting vals is a list of starting vals
# ending vals is a list of ending vals
# num_pts is how many interp values
# quat_vars is the index of any starts of quaternions so they can be SLERPed instead of LERPed
# super_pos_vars is the starting index for 2-pairs of variables which should have an added sine wave (e.g., rubbing)
# super_pos_freq is the frequency for the starting index pairs
# super_pos_amp is the amplitude for the starting index pairs
def interpMultD(starting_vals,ending_vals,num_pts,quat_vars=[], super_pos_vars = [], super_pos_freq=[], super_pos_amp = []):
    vals = []
    
    # LERP ALL VARIABLES
    for ii in range(0,num_pts):
       
        # super is to potentially add a circular motion on top of the LERP motion
        freq_super = 0.0
        amp_super = 0.0
        added_sine = np.zeros((np.shape(starting_vals)))
        for jj in range(0,len(super_pos_vars)):
            amp = super_pos_amp[jj]
            freq = super_pos_freq[jj]
            added_sine[super_pos_vars[jj]] = np.sin(np.pi*float(ii)/num_pts)*amp*np.sin(freq*float(ii)/2*np.pi)
            added_sine[super_pos_vars[jj]+1] = np.sin(np.pi*float(ii)/num_pts)*amp*np.cos(freq*float(ii)/2*np.pi)
        c_i = float(ii)/float(num_pts) # interpolation coefficient
        vals.append(list(c_i*(e-s)+s+a for s,e,a in zip(starting_vals,ending_vals,added_sine)))
    
    vals = np.asarray(vals) # currently,num_samples x num_vars
    
    # SLERP QUATERIONS
    for jj in range(0,len(quat_vars)):
        starting_q = np.array(starting_vals[quat_vars[jj]:quat_vars[jj]+4])
        ending_q = np.array(ending_vals[quat_vars[jj]:quat_vars[jj]+4])
        key_times = [0,num_pts-1]
        key_rots = ScipyR.from_quat(np.concatenate((starting_q.reshape((1,4)),ending_q.reshape((1,4))),axis=0))
        slerper = Slerp(key_times, key_rots)
        
        pts = list(range(0,num_pts))
        slerped = slerper(pts)
        vals[:,quat_vars[jj]:quat_vars[jj]+4] = np.asarray(slerped.as_quat())
    
    return vals.T

# returns num_vars x num_samples


# starting_vals = np.array([0, 1, 2, 0, 0, 0, 1])
# ending_vals = np.array([1, 2, 3, 0, 0, 0, -1])
# out = interpMultD(starting_vals,ending_vals,10,quat_vars=[3])

# print(out)




In [2]:
########################################
# The Sandening                       ##
########################################

starting_surf = [0.1, 0.2]
ending_surf = [0.1, 0.8]
num_passes = 5

import rospkg
rospack = rospkg.RosPack()
uliconfigpath = rospack.get_path('uli_config')


surfaceBSplineFile = uliconfigpath+'/registration_models/IRC_piece1.csv'
surfaceBSpline = BSplineSurface()
surfaceBSpline.loadSurface(surfaceBSplineFile)

segments = []

from scipy.spatial.transform import Rotation as ScipyR
ang_relative_to_surface = 90 # degrees
R_tool_surf = ScipyR.from_euler('ZY',[ang_relative_to_surface,10],degrees=True)

q_ts = R_tool_surf.as_quat()
print(q_ts)

# Part 1: approach the starting point
segment1 = HybridSegment()
segment1.hybrid = False
segment1.num_samples = 60 # 2 seconds
segment1.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
segment1.corrections.append(np.zeros((len(segment1.state_names),segment1.num_samples)))

approach_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(starting_surf[0], starting_surf[1])
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_app = (R_surf * R_tool_surf).as_quat()
if q_app[3]<0:
    q_app*=-1.0

print("QAPP: ",q_app)

starting = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.005, q_app[0], q_app[1], q_app[2], q_app[3], 1.0, 1.0]
orig_vals = interpMultD(starting,ending,segment1.num_samples,quat_vars=[3])


# number of state variables by number of samples

segment1.original_vals = []
segment1.original_vals.append(orig_vals)

segments.append(segment1)


# Part 2: one pass over surface
segment2 = HybridSegment()
segment2.hybrid = True
segment2.surface = surfaceBSpline
segment2.num_samples = 100 # 8 seconds
segment2.state_names = ['u','v','f','theta_qx','theta_qy','theta_qz','theta_qw','delta_s','valve']
segment2.original_vals = []

R_extra_tool = ScipyR.from_euler('y',0,degrees=True)
q_extra = R_extra_tool.as_quat()
corrections = [0.0, 0.0, -5.0, q_extra[0], q_extra[1], q_extra[2], q_extra[3], -1.5, 0.0]
# segment2.corrections.append(interpMultD(corrections,corrections,segment2.num_samples,quat_vars=[3]))
segment2.corrections.append(np.zeros((len(segment2.state_names),segment2.num_samples)))

samps = int(segment2.num_samples / 5)
app_force = -10.0
amp = 0.0
freq = 0.0

waypoint_0 = interpMultD([starting_surf[0], starting_surf[1], app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[ending_surf[0],ending_surf[1], app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],segment2.num_samples,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
segment2.original_vals.append(waypoint_0)

segments.append(segment2)


# # Part 3: move away from the surface
segment3 = HybridSegment()
segment3.hybrid = False
segment3.num_samples = 20 # 2 seconds
segment3.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
segment3.original_vals = []
segment3.corrections.append(np.zeros((len(segment3.state_names),segment3.num_samples)))

retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(ending_surf[0], ending_surf[1])
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_ret = (R_surf * R_tool_surf).as_quat()

starting = [retract_pt[0], retract_pt[1], retract_pt[2]+ 0.005, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 1.0]
ending = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
orig_vals = interpMultD(starting,ending,segment3.num_samples,quat_vars=[3])
segment3.original_vals.append(orig_vals)

segments.append(segment3)


# # Part 4: move from end to start of next pass
segment4 = HybridSegment()
segment4.hybrid = False
segment4.num_samples = 40 # 4 seconds
segment4.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
segment4.original_vals = []
segment4.corrections.append(np.zeros((len(segment4.state_names),segment4.num_samples)))

retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(ending_surf[0], ending_surf[1])
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_ret = (R_surf * R_tool_surf).as_quat()
if q_ret[3]<0:
    q_ret*=-1.0

starting = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
orig_vals = interpMultD(starting,ending,segment4.num_samples,quat_vars=[3])
segment4.original_vals.append(orig_vals)

segments.append(segment4)

for ii in range(0,num_passes-2):
    segments.extend(segments[0:4])
segments.extend(segments[:3])

[-0.06162842  0.06162842  0.70441603  0.70441603]
QAPP:  [-5.30225691e-05 -1.12995201e-01  9.96229704e-03  9.93545587e-01]


In [3]:
############################
# Learn IRC1 model         #
############################
import rospkg
rospack = rospkg.RosPack()
config_dir = rospack.get_path('uli_config')+'/registration_models/'


model = DMPLWRhardcoded(verbose=True, dt=0.1)
learnedSegments = model.learnModel(segments,config_dir+'IRC_piece1.pkl') # second argument is the outfile


-------------------
Model Learning: DMP + LWR Hardcoded
-------------------
DMP for segment  0  of  18
DMP for segment  1  of  18
DMP for segment  2  of  18
DMP for segment  3  of  18
DMP for segment  4  of  18
DMP for segment  5  of  18
DMP for segment  6  of  18
DMP for segment  7  of  18
DMP for segment  8  of  18
DMP for segment  9  of  18
DMP for segment  10  of  18
DMP for segment  11  of  18
DMP for segment  12  of  18
DMP for segment  13  of  18
DMP for segment  14  of  18
DMP for segment  15  of  18
DMP for segment  16  of  18
DMP for segment  17  of  18
DMP for segment  18  of  18
Time to learn model: 19.12792134284973 seconds


In [4]:
############################
# Execute IRC1             #
############################

rospy.init_node('executeROSfromState', anonymous=True)

r_surf = np.array([0.4091875321237706, 0.3659806127623546, 0.6255911510276543, -0.5543099010413256])
t_surf = np.array([0.34226860621143673, -0.22084696598873957, 0.281413347966159])

model.executeModel(learnedSegments=learnedSegments, R_surface = r_surf, t_surface = t_surf, input_type='1dof')

not hybrid
segid:  0  samp: 0.0
segid:  0  samp: 1.0
segid:  0  samp: 2.0
segid:  0  samp: 3.0
segid:  0  samp: 4.0
segid:  0  samp: 5.0
segid:  0  samp: 6.0
segid:  0  samp: 7.0
segid:  0  samp: 8.0
segid:  0  samp: 9.0
segid:  0  samp: 10.0
segid:  0  samp: 11.0
segid:  0  samp: 12.0
segid:  0  samp: 13.0
segid:  0  samp: 14.0
segid:  0  samp: 15.0
segid:  0  samp: 16.0
segid:  0  samp: 17.0
segid:  0  samp: 18.0
segid:  0  samp: 19.0
segid:  0  samp: 20.0
segid:  0  samp: 21.0
segid:  0  samp: 22.0
segid:  0  samp: 23.0
segid:  0  samp: 24.0
segid:  0  samp: 25.0
segid:  0  samp: 26.0
segid:  0  samp: 27.0
segid:  0  samp: 28.0
segid:  0  samp: 29.0
segid:  0  samp: 30.0
segid:  0  samp: 31.0
segid:  0  samp: 32.0
segid:  0  samp: 33.0
segid:  0  samp: 34.0
segid:  0  samp: 35.0
segid:  0  samp: 36.0
segid:  0  samp: 37.0
segid:  0  samp: 38.0
segid:  0  samp: 39.0
segid:  0  samp: 40.0
segid:  0  samp: 41.0
segid:  0  samp: 42.0
segid:  0  samp: 43.0
segid:  0  samp: 44.0
segid:  0

segid:  1  samp: 78.0
segid:  1  samp: 79.0
segid:  1  samp: 80.0
segid:  1  samp: 81.0
segid:  1  samp: 82.0
segid:  1  samp: 83.0
segid:  1  samp: 84.0
segid:  1  samp: 85.0
segid:  1  samp: 86.0
segid:  1  samp: 87.0
segid:  1  samp: 88.0
segid:  1  samp: 89.0
segid:  1  samp: 90.0
segid:  1  samp: 91.0
segid:  1  samp: 92.0
segid:  1  samp: 93.0
segid:  1  samp: 94.0
segid:  1  samp: 95.0
segid:  1  samp: 96.0
segid:  1  samp: 97.0
segid:  1  samp: 98.0
segid:  1  samp: 99.0
not hybrid
segid:  2  samp: 0.0
segid:  2  samp: 1.0
segid:  2  samp: 2.0
segid:  2  samp: 3.0
segid:  2  samp: 4.0
segid:  2  samp: 5.0
segid:  2  samp: 6.0
segid:  2  samp: 7.0
segid:  2  samp: 8.0
segid:  2  samp: 9.0
segid:  2  samp: 10.0
segid:  2  samp: 11.0
segid:  2  samp: 12.0
segid:  2  samp: 13.0
segid:  2  samp: 14.0
segid:  2  samp: 15.0
segid:  2  samp: 16.0
segid:  2  samp: 17.0
segid:  2  samp: 18.0
segid:  2  samp: 19.0
not hybrid
segid:  3  samp: 0.0
segid:  3  samp: 1.0
segid:  3  samp: 2.0
s

segid:  9  samp: 12.0
segid:  9  samp: 13.0
segid:  9  samp: 14.0
segid:  9  samp: 15.0
segid:  9  samp: 16.0
segid:  9  samp: 17.0
segid:  9  samp: 18.0
segid:  9  samp: 19.0
segid:  9  samp: 20.0
segid:  9  samp: 21.0
segid:  9  samp: 22.0
segid:  9  samp: 23.0
segid:  9  samp: 24.0
segid:  9  samp: 25.0
segid:  9  samp: 26.0
segid:  9  samp: 27.0
segid:  9  samp: 28.0
segid:  9  samp: 29.0
segid:  9  samp: 30.0
segid:  9  samp: 31.0
segid:  9  samp: 32.0
segid:  9  samp: 33.0
segid:  9  samp: 34.0
segid:  9  samp: 35.0
segid:  9  samp: 36.0
segid:  9  samp: 37.0
segid:  9  samp: 38.0
segid:  9  samp: 39.0
segid:  9  samp: 40.0
segid:  9  samp: 41.0
segid:  9  samp: 42.0
segid:  9  samp: 43.0
segid:  9  samp: 44.0
segid:  9  samp: 45.0
segid:  9  samp: 46.0
segid:  9  samp: 47.0
segid:  9  samp: 48.0
segid:  9  samp: 49.0
segid:  9  samp: 50.0
segid:  9  samp: 51.0
segid:  9  samp: 52.0
segid:  9  samp: 53.0
segid:  9  samp: 54.0
segid:  9  samp: 55.0
segid:  9  samp: 56.0
segid:  9 

segid:  15  samp: 33.0
segid:  15  samp: 34.0
segid:  15  samp: 35.0
segid:  15  samp: 36.0
segid:  15  samp: 37.0
segid:  15  samp: 38.0
segid:  15  samp: 39.0
not hybrid
segid:  16  samp: 0.0
segid:  16  samp: 1.0
segid:  16  samp: 2.0
segid:  16  samp: 3.0
segid:  16  samp: 4.0
segid:  16  samp: 5.0
segid:  16  samp: 6.0
segid:  16  samp: 7.0
segid:  16  samp: 8.0
segid:  16  samp: 9.0
segid:  16  samp: 10.0
segid:  16  samp: 11.0
segid:  16  samp: 12.0
segid:  16  samp: 13.0
segid:  16  samp: 14.0
segid:  16  samp: 15.0
segid:  16  samp: 16.0
segid:  16  samp: 17.0
segid:  16  samp: 18.0
segid:  16  samp: 19.0
segid:  16  samp: 20.0
segid:  16  samp: 21.0
segid:  16  samp: 22.0
segid:  16  samp: 23.0
segid:  16  samp: 24.0
segid:  16  samp: 25.0
segid:  16  samp: 26.0
segid:  16  samp: 27.0
segid:  16  samp: 28.0
segid:  16  samp: 29.0
segid:  16  samp: 30.0
segid:  16  samp: 31.0
segid:  16  samp: 32.0
segid:  16  samp: 33.0
segid:  16  samp: 34.0
segid:  16  samp: 35.0
segid:  16

In [ ]:
# ########################################
# # IRC BEHAVIOR 1 - with superposition ##
# ########################################

# # Behavior three: Three passes on the IRC surface (slow with corrections)

# import rospkg
# rospack = rospkg.RosPack()
# uliconfigpath = rospack.get_path('uli_config')


# surfaceBSplineFile = uliconfigpath+'/registration_models/IRC_piece1.csv'
# surfaceBSpline = BSplineSurface()
# surfaceBSpline.loadSurface(surfaceBSplineFile)

# segments = []

# from scipy.spatial.transform import Rotation as ScipyR
# ang_relative_to_surface = 90 # degrees
# R_tool_surf = ScipyR.from_euler('z',ang_relative_to_surface,degrees=True)
# q_ts = R_tool_surf.as_quat()


# # Part 1: approach the starting point
# segment1 = HybridSegment()
# segment1.hybrid = False
# segment1.num_samples = 600 # 2 seconds
# segment1.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
# segment1.corrections.append(np.zeros((len(segment1.state_names),segment1.num_samples)))

# approach_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.1, 0.2)
# R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
# q_app = (R_surf * R_tool_surf).as_quat()
# if q_app[3]<0:
#     q_app*=-1.0

# print("QAPP: ",q_app)

# starting = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
# ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.01, q_app[0], q_app[1], q_app[2], q_app[3], 1.0, 1.0]
# orig_vals = interpMultD(starting,ending,segment1.num_samples,quat_vars=[3])


# # number of state variables by number of samples

# segment1.original_vals = []
# segment1.original_vals.append(orig_vals)

# segments.append(segment1)


# # Part 2: one pass over surface
# segment2 = HybridSegment()
# segment2.hybrid = True
# segment2.surface = surfaceBSpline
# segment2.num_samples = 2000 # 8 seconds
# segment2.state_names = ['u','v','f','theta_qx','theta_qy','theta_qz','theta_qw','delta_s','valve']
# segment2.original_vals = []

# R_extra_tool = ScipyR.from_euler('y',5,degrees=True)
# q_extra = R_extra_tool.as_quat()
# corrections = [0.0, 0.0, -5.0, q_extra[0], q_extra[1], q_extra[2], q_extra[3], -1.5, 0.0]
# segment2.corrections.append(interpMultD(corrections,corrections,segment2.num_samples,quat_vars=[3]))


# samps = int(segment2.num_samples / 5)
# app_force = -8.0
# amp = 0.05
# freq = 0.2
# waypoint_0 = interpMultD([0.1, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.1, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
# waypoint_1 = interpMultD([0.1, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.5, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
# waypoint_2 = interpMultD([0.5, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.5, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
# waypoint_3 = interpMultD([0.5, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.9, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
# waypoint_4 = interpMultD([0.9, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.9, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])

# waypoints = np.hstack([waypoint_0,waypoint_1,waypoint_2,waypoint_3,waypoint_4])

# segment2.original_vals.append(waypoints)

# segments.append(segment2)


# # # Part 3: move away from the surface
# segment3 = HybridSegment()
# segment3.hybrid = False
# segment3.num_samples = 200 # 2 seconds
# segment3.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
# segment3.original_vals = []
# segment3.corrections.append(np.zeros((len(segment3.state_names),segment3.num_samples)))

# retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.9, 0.5)
# R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
# q_ret = (R_surf * R_tool_surf).as_quat()

# starting = [retract_pt[0], retract_pt[1], retract_pt[2]+ 0.01, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 1.0]
# ending = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
# orig_vals = interpMultD(starting,ending,segment3.num_samples,quat_vars=[3])
# segment3.original_vals.append(orig_vals)

# segments.append(segment3)


# # # Part 4: move from end to start of next pass
# segment4 = HybridSegment()
# segment4.hybrid = False
# segment4.num_samples = 400 # 4 seconds
# segment4.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
# segment4.original_vals = []
# segment4.corrections.append(np.zeros((len(segment4.state_names),segment4.num_samples)))

# retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.9, 0.5)
# R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
# q_ret = (R_surf * R_tool_surf).as_quat()
# if q_ret[3]<0:
#     q_ret*=-1.0

# starting = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
# ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
# orig_vals = interpMultD(starting,ending,segment4.num_samples,quat_vars=[3])
# segment4.original_vals.append(orig_vals)

# segments.append(segment4)

# segments.extend(segments)
# segments.extend(segments[:3])